<a href="https://colab.research.google.com/github/anishb77/Tensorflow-Projects/blob/main/CARJACK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This time, I want to make an actual project to solve a real problem (obviously there will be better options out there but this will be a fun learning experience). Specifically, I want to create a model that can predict a baseline price of a car based on factors like make and mileage. This means I finally have to learn how to work with real datasets, but I think I can make it work. Also I might remove these notes or change them later.


To start, I need to load in the dataset. I found one on Kaggle that has a high rating and lots of data, and it's being updated once every few months. Unfortunately, that means I can't just download the data and use it like that. I need a script that can get the data for me so that it will still work if the data updates. (Hopefully this project doesn't break in the future)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#I might use a NN but something like regression might just work, will just have to see
import tensorflow as tf
from tensorflow.keras import Layer, Model, Optimizer

I tried reading docs for Kaggle but there's a lot to take in, kaggle had some code and AI had the rest of this:

In [ ]:
!pip install -q kaggle

import kagglehub

# Download latest version
path = kagglehub.dataset_download("austinreese/craigslist-carstrucks-data")

print("Path to dataset files:", path)

import os

# 1. List the files in the path to see the exact CSV name
print(os.listdir(path))

# 2. Load the data (assuming the file is named 'vehicles.csv')
# We use os.path.join to combine the folder path with the filename
csv_path = os.path.join(path, 'vehicles.csv')
car_data = pd.read_csv(csv_path, usecols=["price", "year", "manufacturer", "model", "odometer",
                                          "fuel", "transmission", "cylinders", "drive"]) #should mostly remove data people can lie about

car_data.head()

100%|██████████| 262M/262M [00:12<00:00, 21.7MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/austinreese/craigslist-carstrucks-data/versions/10
['vehicles.csv']


,price,year,manufacturer,model,cylinders,fuel,odometer,transmission,drive
0,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
car_data_clean = car_data.dropna()
car_data_clean.head()

,price,year,manufacturer,model,cylinders,fuel,odometer,transmission,drive
31,15000,2013.0,ford,f-150 xlt,6 cylinders,gas,128000.0,automatic,rwd
32,27990,2012.0,gmc,sierra 2500 hd extended cab,8 cylinders,gas,68696.0,other,4wd
33,34590,2016.0,chevrolet,silverado 1500 double,6 cylinders,gas,29499.0,other,4wd
34,35000,2019.0,toyota,tacoma,6 cylinders,gas,43000.0,automatic,4wd
35,29990,2016.0,chevrolet,colorado extended cab,6 cylinders,gas,17302.0,other,4wd


In [ ]:
car_data_clean.shape

(201141, 9)

Now that my data is somewhat usable, I need to learn as much as I can about it so I can use it properly. I don't know which variables correlate to what and I need to filter out junk that won't help.

In [ ]:
car_data_clean.describe() #I didn't know pandas used scientific notation...

,price,year,odometer
count,2.011410e+05,201141.000000,2.011410e+05
mean,6.075614e+04,2010.586579,1.053813e+05
std,1.205128e+07,9.005131,1.927938e+05
min,0.000000e+00,1900.000000,0.000000e+00
25%,5.500000e+03,2007.000000,5.069600e+04
50%,1.195000e+04,2013.000000,9.715700e+04
75%,2.399500e+04,2016.000000,1.414000e+05
max,3.736929e+09,2022.000000,1.000000e+07


Yeah there are a lot of outliers (like with the min and max), I'll probably have to get rid of them.

In [ ]:
car_data_clean = car_data_clean[(car_data_clean["price"] > 500) & (car_data_clean["price"] < 200000)]
car_data_clean = car_data_clean[(car_data_clean["odometer"] > 0) & (car_data_clean["odometer"] < 400000)]

#this syntax is pretty weird but I guess it makes sense for pd

In [ ]:
car_data_clean.describe()

,price,year,odometer
count,181375.000000,181375.000000,181375.000000
mean,17701.621921,2010.309172,102480.335217
std,14176.321380,9.050045,62281.425618
min,501.000000,1900.000000,1.000000
25%,6995.000000,2007.000000,52278.500000
50%,13500.000000,2012.000000,99364.000000
75%,25800.000000,2016.000000,143947.500000
max,195000.000000,2022.000000,399198.000000


Ok it's time to encode the categorical features now. The ones I need to encode are manufacturer, model,	cylinders,	fuel,	transmission, and drive. Google says one-hot encoders will be fine. I think pandas has a built in way of doing this, but first I'll have to get rid of case sensitivity and white space.

In [ ]:
text_columns = ["manufacturer", "model", "cylinders", "fuel", "transmission", "drive"]

for column in text_columns:
  car_data_clean[column] = car_data_clean[column].str.lower().str.strip()
  #for some reason string functions like this won't work on dataframes but this works

#now to check if it did anything:
print(car_data_clean["manufacturer"].unique())
print(car_data_clean["drive"].unique())

['ford' 'gmc' 'chevrolet' 'toyota' 'jeep' 'nissan' 'honda' 'dodge'
 'chrysler' 'ram' 'mercedes-benz' 'infiniti' 'bmw' 'volkswagen' 'mazda'
 'porsche' 'lexus' 'buick' 'hyundai' 'cadillac' 'acura' 'kia' 'volvo'
 'ferrari' 'pontiac' 'audi' 'mitsubishi' 'subaru' 'lincoln' 'fiat' 'rover'
 'mini' 'jaguar' 'alfa-romeo' 'saturn' 'mercury' 'tesla' 'harley-davidson'
 'datsun' 'aston-martin' 'land rover']
['rwd' '4wd' 'fwd']


In [ ]:
car_data_clean["model"].value_counts() #what...there are so many

,count
model,
f-150,3361
silverado 1500,2676
1500,1960
camry,1532
accord,1481
...,...
a6 3.0t prestige,1
k10 scottsdale,1
g37 x awd,1


Oh my god how do I use any of this? I need the model because it's important to the price, but I need to cut down the values. There's a strategy I found with only using a few of the most common ones and setting the others to a placeholder value like "other". The syntax is crazy but this should do the trick!

In [ ]:
keep_model_values = car_data_clean["model"].value_counts().index[:100]

car_data_clean.loc[~car_data_clean["model"].isin(keep_model_values), "model"] = "other"
#this forced me to struggle with how to make a boolean mask and use the loc function
#I think I'm starting to understand pd better now

In [ ]:
print(car_data_clean["model"].value_counts())

model
other             115364
f-150               3361
silverado 1500      2676
1500                1960
camry               1532
                   ...  
legacy               313
c-class              310
rx 350               304
escalade             301
civic lx             301
Name: count, Length: 101, dtype: int64


It worked! Around half the elements have a model attribute now and most of the clutter is just other. That should be fine for now, it's time to use pd's encoding.

In [ ]:
car_data_encoded = pd.get_dummies(car_data_clean, columns=text_columns)

print(car_data_encoded.head())
car_data_encoded.shape

    price    year  odometer  manufacturer_acura  manufacturer_alfa-romeo  \
31  15000  2013.0  128000.0               False                    False   
32  27990  2012.0   68696.0               False                    False   
33  34590  2016.0   29499.0               False                    False   
34  35000  2019.0   43000.0               False                    False   
35  29990  2016.0   17302.0               False                    False   

    manufacturer_aston-martin  manufacturer_audi  manufacturer_bmw  \
31                      False              False             False   
32                      False              False             False   
33                      False              False             False   
34                      False              False             False   
35                      False              False             False   

    manufacturer_buick  manufacturer_cadillac  ...  fuel_electric  fuel_gas  \
31               False                  Fal

(181375, 164)

Nice, let's go! This should work now. sklearn has a function for splitting data into training and testing, so I'll use that.

In [ ]:
x = car_data_encoded.drop("price", axis=1)
y = car_data_encoded["price"]

In [ ]:
import sklearn

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size=0.2)

Now that I have the train and test sets, I need to test different models and see which ones work better. At the time of writing this I have tensorflow imported just in case I need to make a nn, but I'll try linear regression first as a baseline.

In [ ]:
model = sklearn.linear_model.LinearRegression()

model.fit(x_train, y_train)

LinearRegression()

In [ ]:
predictions = model.predict(x_test)

error = sklearn.metrics.root_mean_squared_error(y_test, predictions)

print(error)

8657.446903441909


Ok This is pretty bad, but it's not that bad. I did some research and there are some models I can try. Since I have so many input columns, I'll try random forests.

In [ ]:
model = sklearn.ensemble.RandomForestRegressor(n_estimators=100, max_depth=20, random_state=67)
#ai recommended these settings, especially random state

model.fit(x_train, y_train)

predictions = model.predict(x_test)
error = sklearn.metrics.root_mean_squared_error(y_test, predictions)
print(error)

5379.510089920661


Honestly, this is much better! The model's trained a little differently every time (obviously) but it should be around an error of 5500. I've consulted AI and it seems like this is basically how good the model can get with data like this. Unfortunately, I couldn't use the "subjective" parts of my data because those are almost definitely lies, but if I had a trustworthy way of getting these it would definitely increase the accuracy by a lot. Maybe if I made another model that could predict the accuracy of an element's "condition", for example, that could be much more usable. For now, though, that will be all! It was really fun making this and experimenting with actual real-life data, and I'll definitely make more stuff soon!